# Data Preprocessing

The goal of this lab is to introduce you to data preprocessing techniques in order to make your data suitable for applying a learning algorithm.

## 1. Handling Missing Values

A common (and very unfortunate) data property is the ocurrence of missing and erroneous values in multiple features in datasets. For this exercise we will be using a data set about abalone snails.
The data set is contained in the Zip file you downloaded from Moodle (abalone.csv).

To determine the age of a abalone snail you have to kill the snail and count the annual
rings. You are told to estimate the age of a snail on the basis of the following attributes:
1. type: male (0), female (1) and infant (2)
2. length in mm
3. width in mm
4. height in mm
5. total weight in grams
6. weight of the meat in grams
7. drained weight in grams
8. weight of the shell in grams
9. number of annual rings (number of rings +1, 5 yields age)

However, the data is incomplete. Missing values are marked with −1.

In [9]:
import pandas as pd
# load data 
df = pd.read_csv("C:\\Users\\ThinkPad\\PycharmProjects\\ML1\\Lab2\\abalone.csv") #Should this not work please use the csv that was part of the zip file.
df.columns=['type','length','width','height','total_weight','meat_weight','drained_weight','shell_weight','num_rings']
df.head()

,type,length,width,height,total_weight,meat_weight,drained_weight,shell_weight,num_rings
0,0,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,-1
1,1,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,9
2,0,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,10
3,2,-1.000,0.255,0.080,0.2050,0.0895,0.0395,0.055,7
4,2,0.425,0.300,0.095,0.3515,0.1410,0.0775,0.120,8


### Exercise 1.1

Compute the mean of of each numeric column and the counts of each categorical column, excluding the missing values.

In [12]:
##################
#INSERT CODE HERE#
##################
from utils.formating import format_title_string

df.replace(-1, pd.NA, inplace=True)

numeric_columns = df.columns.drop('type')
print(df[numeric_columns].mean(skipna=True))
print(df['type'].value_counts())

length            0.523692
width             0.407955
height             0.13961
total_weight      0.828843
meat_weight       0.359263
drained_weight    0.180249
shell_weight      0.238604
num_rings         9.921756
dtype: object
type
0    1500
2    1310
1    1279
Name: count, dtype: int64


### Exercise 1.2

Compute the median of each numeric column,  excluding the missing values.

In [14]:
##################
#INSERT CODE HERE#
##################
df.replace(-1, pd.NA, inplace=True)

numeric_columns = df.columns.drop('type')
print(df[numeric_columns].median(skipna=True))


length              0.545
width               0.425
height               0.14
total_weight      0.80175
meat_weight         0.336
drained_weight     0.1705
shell_weight       0.2335
num_rings             9.0
dtype: object


### Exercise 1.3

Handle the missing values in a way that you find suitable. Think about different ways. Discuss dis-/advantages of your approach. Argue your choices.


In [ ]:
##################
#INSERT CODE HERE#
##################

### Exercise 1.4

Perform Z-score normalization on every column (except the type of course!)

In [15]:
##################
#INSERT CODE HERE#
##################
df.replace(-1, pd.NA, inplace=True)

numeric_columns = df.columns.drop('type')
numeric_df = df[numeric_columns].median(skipna=True)

normalized_df = (numeric_df - numeric_df.mean(skipna=True)) / numeric_df.std(skipna=True)

normalized_df['type'] = df['type']

print(normalized_df.head())

length         -0.298273
width          -0.337543
height         -0.430807
total_weight   -0.214253
meat_weight    -0.366667
dtype: object


## 2. Preprocessing text (Optional)

One possible way to transform text documents into vectors of numeric attributes is to use the TF-IDF representation. We will experiment with this representation using the 20 Newsgroup data set. The data set contains postings on 20 different topics. The classification problem is to decide which of the topics a posting falls into. Here, we will only consider postings about medicine and space.

In [ ]:
from sklearn.datasets import fetch_20newsgroups


categories = ['sci.med', 'sci.space']
raw_data = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
print(f'The index of each category is: {[(i,target) for i,target in enumerate(raw_data.target_names)]}')

Check out some of the postings, might find some funny ones!

In [ ]:
import numpy as np
idx = np.random.randint(0, len(raw_data.data))
print (f'This is a {raw_data.target_names[raw_data.target[idx]]} email.\n')
print (f'There are {len(raw_data.data)} emails.\n')
print(raw_data.data[idx])

Lets pick the first 10 postings from each category

In [ ]:
idxs_med = np.flatnonzero(raw_data.target == 0)
idxs_space = np.flatnonzero(raw_data.target == 1)
idxs = np.concatenate([idxs_med[:10],idxs_space[:10]])
data = np.array(raw_data.data)
data = data[idxs]

<a href="http://www.nltk.org/">NLTK</a> is a toolkit for natural language processing. Take some time to install it and go through this <a href="http://www.slideshare.net/japerk/nltk-in-20-minutes">short tutorial/presentation</a>. (or use e.g. Google colab where the package is prepared already)

The downloaded package below is a tokenizer that divides a text into a list of sentences, by using an unsupervised algorithm to build a model for abbreviation words, collocations, and words that start sentences.

In [ ]:
import nltk
import itertools
nltk.download('punkt')

# Tokenize the sentences into words
tokenized_sentences = [nltk.word_tokenize(sent) for sent in data]
vocabulary_size = 1000
unknown_token = 'unknown'

In [ ]:
# Count the word frequencies
word_freq = nltk.FreqDist(itertools.chain(*tokenized_sentences))
print (f"Found {len(word_freq.items())} unique words tokens.")

In [ ]:
# Get the most common words and build index_to_word and word_to_index vectors
vocab = word_freq.most_common(vocabulary_size-1)
index_to_word = [x[0] for x in vocab]
index_to_word.append(unknown_token)
word_to_index = dict([(w,i) for i,w in enumerate(index_to_word)])
 
print (f"Using vocabulary size {vocabulary_size}." )
print (f"The least frequent word in our vocabulary is '{vocab[-1][0]}' and appeared {vocab[-1][1]} times.")

### Exercise 2.1

Code your own TF-IDF representation function and use it on this dataset. (Don't use code from libraries. Build your own function with Numpy/Pandas). Use the formular TFIDF = TF * (IDF+1). The effect of adding “1” to the idf in the equation above is that terms with zero idf, i.e., terms that occur in all documents in a training set, will not be entirely ignored. The term frequency is the raw count of a term in a document. The inverse document frequency is the natural logarithm of the inverse fraction of the documents that contain the word.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
countvec = CountVectorizer()
df = pd.DataFrame(countvec.fit_transform(data).toarray(), columns=countvec.get_feature_names_out())

def tfidf(df):
    
    ##################
    #INSERT CODE HERE#
    ##################
    return None
    
    
rep = tfidf(df)

# Check if your implementation is correct
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(norm=None, smooth_idf=False, use_idf=True)
X_train = pd.DataFrame(vectorizer.fit_transform(data).toarray(), columns=countvec.get_feature_names_out())
answer=['No','Yes']
epsilon = 0.0001
if rep is None: 
  print (f'Is this implementation correct?\nAnswer: {answer[0]}')
if rep is not None:
  print (f'Is this implementation correct?\nAnswer: {answer[1*np.all((X_train - rep) < epsilon)]}')

In [ ]:
# an example of what to do with these similarities:


# analysis with tf-idf
from sklearn.metrics.pairwise import cosine_similarity

similiarities = cosine_similarity(rep, rep) # measure of the similarity of the direction of two vectors

In [ ]:
np.fill_diagonal(similiarities, 0)
max_ind = np.unravel_index(similiarities.argmax(), similiarities.shape)
similiarities[max_ind] # highest similarity of two documents